**ЛАБОРАТОРНАЯ РАБОТА 3. МЕТРИЧЕСКИЕ МЕТОДЫ 
КЛАССИФИКАЦИИ**

**В качестве набора ресурсов будет использоваться Wholesale customers data.csv**

**Описание набора:**

**Набор данных относится к клиентам оптового дистрибьютора. Он включает ежегодные расходы в денежных единицах (м.у.) на различные категории товаров**

**Перечень признаков:**

**Channel - клиенты Канал - Horeca (отель/ Ресторан/ кафе) или розничный канал (Номинальный)**

**Region - регион клиентов - Лиссабон, Порту или другой (номинальный)**

**FRESH - ежегодные расходы (млн. у.е.) на свежие продукты (непрерывно)**

**MILK - ежегодные расходы (млн.у.е.) на молочные продукты (непрерывно)**

**GROCERY - ежегодные расходы (млн.у.е.) на продуктовые продукты (непрерывно)**

**FROZEN - ежегодные расходы (млн.у.е.) на замороженные продукты (непрерывно)**

**DETERGENTS_PAPER - ежегодные расходы (млн.у.е.) на моющие средства и бумажные изделия (непрерывно)**

**DELICATESSEN - ежегодные расходы (млн.у.е.) на деликатессы (непрерывные)**

**Описательная статистика: (Минимальное, максимальное, среднее значение, стандартное отклонение)**

**(Minimum, Maximum, Mean, Std. Deviation)**

**FRESH (	3, 112151, 12000.30, 12647.329)**

**MILK	(55, 73498, 5796.27, 7380.377)**

**GROCERY	(3, 92780, 7951.28, 9503.163)**

**FROZEN	(25, 60869, 3071.93, 4854.673)**

**DETERGENTS_PAPER (3, 40827, 2881.49, 4767.854)**

**DELICATESSEN (3, 47943, 1524.87, 2820.106)**

**REGION	Frequency**

**Lisbon	77**

**Oporto	47**

**Other Region	316**

**Total	440**

**CHANNEL	Frequency**
**Horeca	298**

**Retail	142**

**Total	440**




**Для выполнения 3 лабораторной необходимо:**

Выполните построение модели классификации на основе метода 
ближайших соседей. В ходе решения задачи необходимо решить следующие 
подзадачи:

2.1 Построение классификатора с заданием K (количества ближайших 
соседей) пользователем;

2.2 Вычисление оценки hold-out для различнх значений K, а также для 
различных долей обучающей и тестирующей подвыборок;

2.3 Вычисление оценки cross validation для различных значений K, а 
также для различных значений fold (количества подмножеств при кроссвалидации).

2.4 Вычислите оптимальные значения K. Обоснуйте свой выбор. 
Продемонстрируйте использование полученного классификатора

In [1]:
# Импортируем необходимые библиотеки
import warnings  # Для обработки предупреждений
import numpy as np  # Для работы с массивами и математическими операциями
import pandas as pd  # Для работы с табличными данными (DataFrame)
from sklearn.model_selection import train_test_split, cross_val_score  # Для разбиения данных и кросс-валидации
from sklearn.neighbors import KNeighborsClassifier  # Для использования классификатора K ближайших соседей
from sklearn.metrics import accuracy_score  # Для вычисления точности модели
from sklearn.preprocessing import StandardScaler  # Для масштабирования признаков
from collections import Counter  # Для подсчета количества элементов в классе

# Игнорируем предупреждения, чтобы они не мешали выполнению кода
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    # Загрузка данных из CSV файла
    file_path = "Wholesale customers data.csv"  
    data = pd.read_csv(file_path)  # Загрузка данных в DataFrame
    
    # Вывод первых строк данных для ознакомления
    print("Первые строки данных:")
    print(data.head())  # Показываем первые 5 строк данных для проверки

    # Проверка на пропущенные значения
    if data.isnull().sum().any():  # Проверяем, есть ли пропущенные значения в данных
        print("\nДанные содержат пропущенные значения. Они будут заполнены средним значением.")
        data.fillna(data.mean(), inplace=True)  # Если есть пропущенные значения, заменяем их средним значением

    # Разделяем данные на признаки (X) и целевую переменную (y)
    X = data.drop("Region", axis=1)  # Все колонки, кроме "Region", будут признаками
    y = data["Region"]  # Целевая переменная — это колонка "Region"

    # Масштабируем данные для приведения всех признаков к одному масштабу
    scaler = StandardScaler()  # Инициализируем объект для масштабирования данных
    X_scaled = scaler.fit_transform(X)  # Применяем масштабирование ко всем признакам

    # Анализ распределения классов целевой переменной (Region)
    class_counts = Counter(y)  # Подсчитываем количество объектов в каждом классе
    min_samples = min(class_counts.values())  # Находим минимальное количество объектов в классе
    n_splits = min(5, min_samples)  # Для кросс-валидации количество разбиений не должно превышать минимальное количество объектов в классе
    print(f"\nРаспределение классов: {class_counts}")
    print(f"Минимальное число образцов в классе: {min_samples}")
    print(f"Количество разбиений для кросс-валидации: {n_splits}")

    # Создаём DataFrame для хранения результатов
    results_df = pd.DataFrame(columns=["test_size", "k", "accuracy", "cv_accuracy"])  # Определяем колонки для хранения данных

    # Диапазон значений для параметра k (количество ближайших соседей) и долей тестовой выборки
    k_values = range(1, 31)  # Значения k от 1 до 30
    test_sizes = [0.1, 0.2, 0.3, 0.4, 0.5]  # Разные пропорции для тестовой выборки

    # Основной цикл по значениям test_size и k для обучения модели
    for test_size in test_sizes:  # Перебираем разные доли для тестовой выборки
        for k in k_values:  # Перебираем разные значения для k (количество соседей)
            # Разделяем данные на обучающую и тестовую выборки
            X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=test_size, random_state=42)
            
            # Создаём классификатор KNN с текущим значением k
            knn = KNeighborsClassifier(n_neighbors=k)
            knn.fit(X_train, y_train)  # Обучаем модель на обучающей выборке
            
            # Предсказание на тестовой выборке
            y_pred = knn.predict(X_test)  # Получаем предсказания для тестовой выборки
            accuracy = accuracy_score(y_test, y_pred)  # Вычисляем точность модели на тестовых данных
            
            # Кросс-валидация для вычисления стабильности модели
            cv_scores = cross_val_score(knn, X_scaled, y, cv=n_splits)  # Кросс-валидация с заданным количеством разбиений
            cv_mean_accuracy = np.mean(cv_scores)  # Среднее значение точности кросс-валидации
            
            # Сохраняем результаты для текущих параметров
            results_df = pd.concat([
                results_df,  # Добавляем новые результаты к предыдущим
                pd.DataFrame({
                    "test_size": [test_size],
                    "k": [k],
                    "accuracy": [accuracy],
                    "cv_accuracy": [cv_mean_accuracy]
                })
            ], ignore_index=True)

    # Сортируем результаты по точности модели
    results_df = results_df.sort_values(by="accuracy", ascending=False)  # Сортировка по убыванию точности

    # Выводим топ-10 лучших результатов
    print("\nТоп-10 лучших результатов:")
    print(results_df.head(10))  # Показываем 10 лучших результатов по точности


Первые строки данных:
   Channel  Region  Fresh  Milk  Grocery  Frozen  Detergents_Paper  Delicassen
0        2       3  12669  9656     7561     214              2674        1338
1        2       3   7057  9810     9568    1762              3293        1776
2        2       3   6353  8808     7684    2405              3516        7844
3        1       3  13265  1196     4221    6404               507        1788
4        2       3  22615  5410     7198    3915              1777        5185

Распределение классов: Counter({3: 316, 1: 77, 2: 47})
Минимальное число образцов в классе: 47
Количество разбиений для кросс-валидации: 5

Топ-10 лучших результатов:
    test_size   k  accuracy  cv_accuracy
75        0.3  16  0.840909     0.718182
88        0.3  29  0.840909     0.718182
86        0.3  27  0.840909     0.718182
85        0.3  26  0.840909     0.718182
84        0.3  25  0.840909     0.718182
83        0.3  24  0.840909     0.718182
81        0.3  22  0.840909     0.718182
80      

test_size - доля данных, которая выделяется для тестирования модели. 
Например, test_size=0.3 означает, что 30% данных используется для тестирования, а 70% — для обучения.

k - количество ближайших соседей, используемых в алгоритме K-ближайших соседей (KNN). Это ключевой параметр, определяющий, сколько ближайших точек учитывается при классификации.

accuracy - точность модели на тестовой выборке. Она показывает долю правильно классифицированных объектов среди всех тестовых примеров.

cv_accuracy - средняя точность модели, вычисленная с помощью кросс-валидации (перекрёстной проверки). Это показатель стабильности и обобщающей способности модели на различных подвыборках данных.

**Вывод**

**В этой работе мы применили алгоритм KNN для классификации, провели масштабирование данных, кросс-валидацию и выбрали лучшие параметры модели.**

**Классификатор на основе метода ближайших соседей (KNN) показал хорошую точность предсказания, с наилучшими результатами при значении K=16 и размере тестовой выборки 30%, где точность составила 84%, что подтверждается стабильными результатами кросс-валидации.**